<a href="https://colab.research.google.com/github/ll1195831146/Infor7374-AI/blob/master/Assignment3/H2O_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import json

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

from __future__ import absolute_import, division, print_function
keras.__version__

Using TensorFlow backend.


'2.2.4'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
os.chdir(os.path.join(os.getcwd(),'drive/My Drive'))
print(os.getcwd())

/content/drive/My Drive


In [0]:
with open('Final_train.json') as train_f:
   train_data = json.load(train_f)
with open('Final_test.json') as test_f:
   test_data = json.load(test_f)

In [0]:
train_data = pd.DataFrame(train_data)
train_data['sentiment'] = train_data['sentiment'].str.lower()
train_data = train_data[train_data.sentiment != 'neutra;']
train_data = train_data[train_data.sentiment != 'postive']
test_data = pd.DataFrame(test_data)
test_data['sentiment'] = test_data['sentiment'].str.lower()
test_data = test_data[test_data.sentiment != 'neutra;']
test_data = test_data[test_data.sentiment != 'postive']

In [0]:
text_train = train_data['text']
label_train = train_data['sentiment']
text_test = test_data['text']
label_test = test_data['sentiment']

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
x_train = vect.fit(text_train).transform(text_train)
x_test = vect.transform(text_test)

print("Vocabulary size: {}".format(len(vect.vocabulary_)))
print("X_train:\n{}".format(repr(x_train)))
print("X_test: \n{}".format(repr(x_test)))

feature_names = vect.get_feature_names()
print("Number of features: {}".format(len(feature_names)))

Vocabulary size: 4571
X_train:
<1048x4571 sparse matrix of type '<class 'numpy.int64'>'
	with 43129 stored elements in Compressed Sparse Row format>
X_test: 
<264x4571 sparse matrix of type '<class 'numpy.int64'>'
	with 12084 stored elements in Compressed Sparse Row format>
Number of features: 4571


In [0]:
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
# Use sklearn utility to convert label strings to numbered index
encoder = LabelEncoder()
encoder.fit(label_train)
y_train = encoder.transform(label_train)
y_test = encoder.transform(label_test)

In [13]:
import h2o
h2o.init()
from h2o.estimators.word2vec import H2OWord2vecEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "10.0.2" 2018-07-17; OpenJDK Runtime Environment (build 10.0.2+13-Ubuntu-1ubuntu0.18.04.4); OpenJDK 64-Bit Server VM (build 10.0.2+13-Ubuntu-1ubuntu0.18.04.4, mixed mode)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpfx0ow5b0
  JVM stdout: /tmp/tmpfx0ow5b0/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpfx0ow5b0/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.6
H2O cluster version age:,"7 days, 1 hour and 29 minutes"
H2O cluster name:,H2O_from_python_unknownUser_0lfxxl
H2O cluster total nodes:,1
H2O cluster free memory:,2.938 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"accepting new members, healthy"


In [18]:
reviews = h2o.import_file("Complete.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [19]:
reviews.head()

C1,sentiment,text
0,positive,"Thank you, Ellen. We have a strong 2018, with total revenues of $136.8 billion, up 23% over 2017, reflecting the benefit of our ongoing investments to deliver exceptional experiences for users and compelling returns for our advertisers, partners and enterprise customers."
1,positive,"Stock-based compensation totaled $2.3 billion. Headcount at the end of the quarter was 98,771, up 4,399 from last quarter. Consistent with prior quarters, the majority of new hires were engineers and product managers. In terms of product areas, the most sizable head count increases were in Cloud for both technical and sales roles."
10,negative,"For the full-year 2018, Other Bets revenues were $595 million, up 25% versus 2017 primarily generated by Fiber and Verily. Operating loss for Other Bets was $3.4 billion for the full-year 2018 versus an operating loss of $2.7 billion in 2017. Other Bets accrued CapEx was $181 million, down from $493 million in 2017 primarily reflecting investments in Fiber."
100,neutral,"Great. Thank you. I just wanted to follow up on India, just bigger picture, Brian, do the new policies change your view at all of the attractiveness or the potential of operating in India and how do you think about your investment strategy there in the near term? And then also just hoping you could talk about some of the impact in the US and UK from free shipping during the fourth quarter holidays?"
1000,neutral,Operator
1001,neutral,"At this time, there are no further questions. Please continue with any closing remarks."
1002,neutral,Stephen Scherr
1003,positive,"Okay. Since there are no more questions, we would like to take a moment to thank everybody for joining the call. On behalf of our senior management team, we hope to see many of you in the coming months. If any additional questions arise in the meantime, please don't hesitate to reach out to Heather. Otherwise, enjoy the rest of your day, and we look forward to speaking with you on our first quarter call in April. Thank you."
1004,neutral,Operator
1005,neutral,"Ms. Miner, you may begin your conference."


**Train the Model**

In [0]:
from h2o.automl import H2OAutoML

In [0]:
X = ['text']
y = "sentiment"

In [0]:
reviews[X] = reviews[X].asfactor()
reviews[y] = reviews[y].asfactor()

In [32]:
aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=X, y=y, training_frame=reviews)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [33]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,mean_per_class_error,logloss,rmse,mse
GBM_3_AutoML_20190320_222459,0.575257,1.00263,0.614912,0.378116
StackedEnsemble_BestOfFamily_AutoML_20190320_222459,0.576643,0.891261,0.570582,0.325564
GBM_4_AutoML_20190320_222459,0.580169,1.00139,0.614542,0.377662
StackedEnsemble_AllModels_AutoML_20190320_222459,0.580867,0.898538,0.573022,0.328355
GBM_2_AutoML_20190320_222459,0.585912,1.00351,0.615014,0.378243
DRF_1_AutoML_20190320_222459,0.586274,25.4746,0.861694,0.742517
GBM_1_AutoML_20190320_222459,0.591442,1.00806,0.616403,0.379952
XRT_1_AutoML_20190320_222459,0.600998,6.26772,0.604651,0.365602
DeepLearning_grid_1_AutoML_20190320_222459_model_1,0.610937,1.27363,0.632715,0.400328
GBM_grid_1_AutoML_20190320_222459_model_2,0.615114,1.39495,0.607832,0.36946


In [34]:
# The leader model is stored here
aml.leader

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_3_AutoML_20190320_222459


ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 0.003479620781523639
RMSE: 0.05898831054983385
LogLoss: 0.04024646567686367
Mean Per-Class Error: 0.002287138211001661
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



negative,neutral,positive,Error,Rate
139.0,0.0,0.0,0.0,0 / 139
1.0,630.0,1.0,0.0031646,2 / 632
0.0,2.0,539.0,0.0036969,2 / 541
140.0,632.0,540.0,0.0030488,"4 / 1,312"


Top-3 Hit Ratios: 


k,hit_ratio
1,0.9969512
2,1.0
3,1.0



ModelMetricsMultinomial: gbm
** Reported on cross-validation data. **

MSE: 0.37811632960475855
RMSE: 0.6149116437381541
LogLoss: 1.002634385476192
Mean Per-Class Error: 0.5752572092399784
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



...

,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
accuracy,0.514498,0.0240691,0.4638783,0.5437263,0.5496183,0.5305343,0.4847328
err,0.4855020,0.0240691,0.5361217,0.4562738,0.4503817,0.4694656,0.5152672
err_count,127.4,6.3655324,141.0,120.0,118.0,123.0,135.0
logloss,1.0025539,0.0337019,1.0729526,1.037757,0.9391165,0.9926461,0.9702972
max_per_class_error,0.8782043,0.0994082,0.6525424,0.9642857,1.0,1.0,0.7741935
mean_per_class_accuracy,0.4224973,0.0114296,0.4520261,0.4169557,0.4087043,0.4084142,0.426386
mean_per_class_error,0.5775027,0.0114296,0.5479739,0.5830443,0.5912957,0.5915858,0.573614
mse,0.3780968,0.0083848,0.394388,0.3874008,0.3607019,0.3767239,0.3712695
r2,0.1067800,0.0294355,0.1075239,0.0810027,0.1570482,0.0433976,0.1449278
rmse,0.6148201,0.0068226,0.6280032,0.6224153,0.6005846,0.6137784,0.6093189


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error
,2019-03-20 22:27:02,3.634 sec,0.0,0.6666667,1.0986123,0.5975610
,2019-03-20 22:27:02,3.735 sec,5.0,0.4231455,0.5491733,0.0068598
,2019-03-20 22:27:02,3.839 sec,10.0,0.2644951,0.3026707,0.0038110
,2019-03-20 22:27:02,3.941 sec,15.0,0.1658151,0.1740057,0.0030488
,2019-03-20 22:27:03,4.046 sec,20.0,0.1072484,0.1030964,0.0030488
,2019-03-20 22:27:03,4.156 sec,25.0,0.0751747,0.0632271,0.0030488
,2019-03-20 22:27:03,4.262 sec,30.0,0.0589883,0.0402465,0.0030488


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
text,3553.1425781,1.0,1.0
